# Project 1: Web scraping and basic summarization
*University of Ljubljana, Faculty for computer and information science* <br />
*Course: Introduction to data science*

In this Project you need to implement missing parts of this Jupyter notebook. All the code in the notebook must be reproducible and runnable, so include instructions for the environment setup or other specifics needed to run the notebook. The overview of the repository and setup should be put to README.md

The idea of this Project is to automatically retrieve structured data from pages [rtvslo.si](https://www.rtvslo.si) or [24ur.com](https://www.24ur.com). 

## Environment setup

Write instructions how to setup the environment to run this notebook, which libraries are installed, etc. Also provide installation sources.

`TODO: environment setup description`

In [1]:
# Load all the libraries needed for running the code chunks below

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
import os
from selenium.webdriver.common.by import By
import json
from helper_f import links,  get_article_data,  get_article_comments,  edit_and_join, get_data_skit, get_data_dostopno, get_data_enostavno, make_json_format, change_day_published


19


## Web scraping



JSON Schema:

```
[
  {
    "author": ["author_1", "author_2",...],
    "day_published": "DD.MM.YYYY",
    "changed_later": "YES"/"NO", 
    "title": "article_title",
    "subtitle": "article_subtitle",
    "tags": ["tag_1", "tag_2",...],
    "section_tag": "section_tag"
    "content": "article_text",
    "comments": [
        {
            "user": "user_name",
            "date_hour": ["DD.MM.YYYY"; "MM:HH"],
            "grade": comment_grade(as number),
            "reply": "YES"/"NO",
            "comment": "comment_text",

        },...
    ],
    "hour_published": "MM:HH"
    
  }, 
  {
    ...
]




`TODO: definition and short description of JSON data schema of the extracted data.`

Proposed structure of the implementation is below. You can also import code from the accompanying *.py* files in the repository, so the code is clearer or organize code completely by yourself.

In [2]:
data = []
data_urls = [] #normal rtvslo articles with no SKIT, ENOSTAVNO, DOSTOPNO articles

enostavno = []
skit = []
dostopno = []
i = 0
print("Started scraping")
file = open('article_urls.txt', 'r', encoding='utf-8')
lines = file.readlines()
for line in lines: #urls 
    if "/enostavno/" in line:
        enostavno.append(line[:-1])
    elif "/skit/" in line:
        skit.append(line[:-1]) #\n has to be removed
    elif "/dostopno/" in line:
        dostopno.append(line[:-1])
    else: #normal ones
        data_urls.append(line[:-1])
        #print("Starting to write url number ", )

        
u = len(data_urls)
s = len(skit)
d = len(dostopno)
e = len(enostavno)
        
for j in range(u): #len data_urls
    if os.path.isfile('json/{}.json'.format(j)): #check if article data was already scraped
        pass
    else: #get article and comments data

        (authors, title, subtitle, date, hour, change, article_tags, section_tag, text) = get_article_data(data_urls[j])
        line_data = make_json_format(authors, title, subtitle, date, hour, change, article_tags, section_tag, text, comments)
        
        data.append(line_data)


        #write to seperate .json file
        with open("json/{}.json".format(j), "w", encoding='utf-8') as outfile:
            json.dump(line_data, outfile, ensure_ascii=False)
            
print("END OF MAIN SCRAPING!!!")

#scrapaj še za SKIT, DOSTOPNO, ENOSTAVNO in zapiši v json in potem naredi json edited. 

for i in range(s): #len skit
    if os.path.isfile('json/{}.json'.format(u + i)): #check if article data was already scraped
        pass
    else:
        (authors, title, subtitle, date, hour, change, article_tags, section_tag, text, comments) = get_data_skit(skit[i])
        line_data = make_json_format(authors, title, subtitle, date, hour, change, article_tags, section_tag, text, comments)
        
        data.append(line_data)


        #write to seperate .json file
        with open("json/{}.json".format(i + u), "w", encoding='utf-8') as outfile:
            json.dump(line_data, outfile, ensure_ascii=False)
            
print("END OF SKIT SCRAPING!!!")




for i in range(d):
    if os.path.isfile('json/{}.json'.format(u + s + i)): #check if article data was already scraped
        pass
    else:
        (authors, title, subtitle, date, hour, change, article_tags, section_tag, text, comments) = get_data_dostopno(dostopno[i])
        line_data = make_json_format(authors, title, subtitle, date, hour, change, article_tags, section_tag, text, comments)
        
        data.append(line_data)


        #write to seperate .json file
        with open("json/{}.json".format(i + u + s), "w", encoding='utf-8') as outfile:
            json.dump(line_data, outfile, ensure_ascii=False)
            
print("END OF DOSTOPNO SCRAPING!!!")


for i in range(e):
    if os.path.isfile('json/{}.json'.format(u + s + i + d)): #check if article data was already scraped
        pass
    else:
        (authors, title, subtitle, date, hour, change, article_tags, section_tag, text, comments) = get_data_enostavno(enostavno[i])
        line_data = make_json_format(authors, title, subtitle, date, hour, change, article_tags, section_tag, text, comments)
        
        data.append(line_data)


        #write to seperate .json file
        with open("json/{}.json".format(i + u + s + d), "w", encoding='utf-8') as outfile:
            json.dump(line_data, outfile, ensure_ascii=False)
            
print("END OF ENOSTAVNO SCRAPING!!!")



        
    

Started scraping
END OF MAIN SCRAPING!!!
Starting to get data from article:  SKIT
Starting to get data from article:  SKIT
END OF SKIT SCRAPING!!!
END OF DOSTOPNO SCRAPING!!!
END OF ENOSTAVNO SCRAPING!!!


In [3]:
edit_and_join() #fix typos and join all single article json files into one big json file with all data
print("Json files fixed and joined into data.json file.")

Json files fixed and joined into data.json file.


In [4]:
bla = "23. marec 2021".split(" ")
blabla = "23. marec 2021".replace('.','')
blabla

'23 marec 2021'

In [5]:
bla[1]= "5"
blabla

'23 marec 2021'

In [6]:
change_date_published("")

NameError: name 'change_date_published' is not defined

In [ ]:
# Implement news article page parsing

def parse_news_article(url):
    ...
    return news_article_data

In [ ]:
# Implement search results page parsing 

def parse_search_results(url):
    ...
    return results, next_page_url

In [ ]:
# Implement parsing, data merging and final representation

def search():
    ...
    return data

In [ ]:
# Main program and data export into a JSON

MAIN_URL = "https://www.rtvslo.si"

data = search()
# Save data to UTF-8 encoded JSON
...

## Basic summarization

Prepare and show at least five basic visualizations of the extracted data as presented in the chapter *Summarizing data - the basics* of the course's e-book. Explain each visualization of the data.

In [ ]:
# Read data from JSON

data = ...

### Visualization 1

`TODO: name the visualization and describe it`

In [ ]:
# Visualization 1 code

...

### Visualization 2

`TODO: name the visualization and describe it`

In [ ]:
# Visualization 2 code

...

### Visualization 3

`TODO: name the visualization and describe it`

In [ ]:
# Visualization 3 code

...

### Visualization 4

`TODO: name the visualization and describe it`

In [ ]:
# Visualization 4 code

...

### Visualization 5

`TODO: name the visualization and describe it`

In [ ]:
# Visualization 5 code

...